In [2]:
import numpy as np
from einops import rearrange

In [59]:
a = np.array([[1.2,2,3], [4,5,6],[7,8,9]])
b = np.array([[1,2,3], [8,49,10]])
c = np.array([[1,2,300], [8,49,10]])
# d = np.max(np.maximum(a, b, c))
# c = np.max(a)


In [19]:
x = np.array([1,2,3,3,3,6,7])
a,b,c = x[2:5]

In [20]:
unique = []
unique.append(x[0])
for numb in x[1:]:
    prev_unique = unique[-1]
    if prev_unique != numb:
        unique.append(numb)

In [2]:
a = 7
bin = 3
if a < 0:
    a += bin
if a >= bin:
    a -= bin
print(a)

4
